## All you need is love… And a pet!

<img src="img/dataset-cover.jpg" width="920">

Here we are going to build a classifier to predict whether an animal from an animal shelter will be adopted or not (aac_intakes_outcomes.csv, available at: https://www.kaggle.com/aaronschlegel/austin-animal-center-shelter-intakes-and-outcomes/version/1#aac_intakes_outcomes.csv). You will be working with the following features:

1. *animal_type:* Type of animal. May be one of 'cat', 'dog', 'bird', etc.
2. *intake_year:* Year of intake
3. *intake_condition:* The intake condition of the animal. Can be one of 'normal', 'injured', 'sick', etc.
4. *intake_number:* The intake number denoting the number of occurrences the animal has been brought into the shelter. Values higher than 1 indicate the animal has been taken into the shelter on more than one occasion.
5. *intake_type:* The type of intake, for example, 'stray', 'owner surrender', etc.
6. *sex_upon_intake:* The gender of the animal and if it has been spayed or neutered at the time of intake
7. *age_upon\_intake_(years):* The age of the animal upon intake represented in years
8. *time_in_shelter_days:* Numeric value denoting the number of days the animal remained at the shelter from intake to outcome.
9. *sex_upon_outcome:* The gender of the animal and if it has been spayed or neutered at time of outcome
10. *age_upon\_outcome_(years):* The age of the animal upon outcome represented in years
11. *outcome_type:* The outcome type. Can be one of ‘adopted’, ‘transferred’, etc.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp
from itertools import combinations 
import ast
from sklearn.linear_model import LogisticRegression
import seaborn as sn
%matplotlib inline

data_folder = './data/'

### A) Load the dataset and convert categorical features to a suitable numerical representation (use dummy-variable encoding). 
- Split the data into a training set (80%) and a test set (20%). Pair each feature vector with the corresponding label, i.e., whether the outcome_type is adoption or not. 
- Standardize the values of each feature in the data to have mean 0 and variance 1.

The use of external libraries is not permitted in part A, except for numpy and pandas. 
You can drop entries with missing values.

In [17]:
columns = ['animal_type', 'intake_year', 'intake_condition', 'intake_number', 'intake_type', 'sex_upon_intake', \
          'age_upon_intake_(years)', 'time_in_shelter_days', 'sex_upon_outcome', 'age_upon_outcome_(years)', \
          'outcome_type']
original_data = pd.read_csv(data_folder+'aac_intakes_outcomes.csv', usecols=columns)
original_data.head()

,outcome_type,sex_upon_outcome,age_upon_outcome_(years),animal_type,intake_condition,intake_type,sex_upon_intake,age_upon_intake_(years),intake_year,intake_number,time_in_shelter_days
0,Return to Owner,Neutered Male,10.0,Dog,Normal,Stray,Neutered Male,10.0,2017,1.0,0.588194
1,Return to Owner,Neutered Male,7.0,Dog,Normal,Public Assist,Neutered Male,7.0,2014,2.0,1.259722
2,Return to Owner,Neutered Male,6.0,Dog,Normal,Public Assist,Neutered Male,6.0,2014,3.0,1.113889
3,Transfer,Neutered Male,10.0,Dog,Normal,Owner Surrender,Neutered Male,10.0,2014,1.0,4.970139
4,Return to Owner,Neutered Male,16.0,Dog,Injured,Public Assist,Neutered Male,16.0,2013,1.0,0.119444


In [18]:
original_data.dropna(inplace=True)

In [19]:
data_features = original_data.copy()
data_features['adopted'] = data_features.outcome_type.apply(lambda r: 1 if r=='Adoption' else 0)
data_features.drop("outcome_type", axis = 1, inplace=True)

In [22]:
# Dummy encoding
dummy_data = pd.get_dummies(data_features, columns=['sex_upon_outcome','animal_type','intake_condition','intake_type','sex_upon_intake',])
dummy_data

,age_upon_outcome_(years),age_upon_intake_(years),intake_year,intake_number,time_in_shelter_days,adopted,sex_upon_outcome_Intact Female,sex_upon_outcome_Intact Male,sex_upon_outcome_Neutered Male,sex_upon_outcome_Spayed Female,...,intake_type_Euthanasia Request,intake_type_Owner Surrender,intake_type_Public Assist,intake_type_Stray,intake_type_Wildlife,sex_upon_intake_Intact Female,sex_upon_intake_Intact Male,sex_upon_intake_Neutered Male,sex_upon_intake_Spayed Female,sex_upon_intake_Unknown
0,10.000000,10.000000,2017,1.0,0.588194,0,False,False,True,False,...,False,False,False,True,False,False,False,True,False,False
1,7.000000,7.000000,2014,2.0,1.259722,0,False,False,True,False,...,False,False,True,False,False,False,False,True,False,False
2,6.000000,6.000000,2014,3.0,1.113889,0,False,False,True,False,...,False,False,True,False,False,False,False,True,False,False
3,10.000000,10.000000,2014,1.0,4.970139,0,False,False,True,False,...,False,True,False,False,False,False,False,True,False,False
4,16.000000,16.000000,2013,1.0,0.119444,0,False,False,True,False,...,False,False,True,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79667,0.038356,0.038356,2018,1.0,0.077083,0,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True
79668,2.000000,2.000000,2018,1.0,0.053472,0,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
79669,1.000000,1.000000,2018,1.0,0.047917,0,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
79670,0.821918,0.410959,2018,1.0,1.762500,0,False,True,False,False,...,False,False,False,True,False,False,True,False,False,False


In [23]:
# Standardize
dummy_data =( dummy_data - dummy_data.mean() ) / dummy_data.std()
dummy_data

,age_upon_outcome_(years),age_upon_intake_(years),intake_year,intake_number,time_in_shelter_days,adopted,sex_upon_outcome_Intact Female,sex_upon_outcome_Intact Male,sex_upon_outcome_Neutered Male,sex_upon_outcome_Spayed Female,...,intake_type_Euthanasia Request,intake_type_Owner Surrender,intake_type_Public Assist,intake_type_Stray,intake_type_Wildlife,sex_upon_intake_Intact Female,sex_upon_intake_Intact Male,sex_upon_intake_Neutered Male,sex_upon_intake_Spayed Female,sex_upon_intake_Unknown
0,2.709408,2.727906,1.200104,-0.278015,-0.387947,-0.853951,-0.36369,-0.373009,1.347460,-0.687128,...,-0.056221,-0.482173,-0.258562,0.651339,-0.213054,-0.650811,-0.682500,2.295430,-0.403480,-0.305102
1,1.674938,1.690967,-1.102049,1.914751,-0.371836,-0.853951,-0.36369,-0.373009,1.347460,-0.687128,...,-0.056221,-0.482173,3.867498,-1.535280,-0.213054,-0.650811,-0.682500,2.295430,-0.403480,-0.305102
2,1.330114,1.345321,-1.102049,4.107518,-0.375335,-0.853951,-0.36369,-0.373009,1.347460,-0.687128,...,-0.056221,-0.482173,3.867498,-1.535280,-0.213054,-0.650811,-0.682500,2.295430,-0.403480,-0.305102
3,2.709408,2.727906,-1.102049,-0.278015,-0.282819,-0.853951,-0.36369,-0.373009,1.347460,-0.687128,...,-0.056221,2.073918,-0.258562,-1.535280,-0.213054,-0.650811,-0.682500,2.295430,-0.403480,-0.305102
4,4.778349,4.801783,-1.869433,-0.278015,-0.399193,-0.853951,-0.36369,-0.373009,1.347460,-0.687128,...,-0.056221,-0.482173,3.867498,-1.535280,-0.213054,-0.650811,-0.682500,2.295430,-0.403480,-0.305102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79667,-0.725600,-0.715299,1.967488,-0.278015,-0.400209,-0.853951,-0.36369,-0.373009,-0.742128,-0.687128,...,-0.056221,-0.482173,-0.258562,0.651339,-0.213054,-0.650811,-0.682500,-0.435643,-0.403480,3.277552
79668,-0.049180,-0.037265,1.967488,-0.278015,-0.400775,-0.853951,-0.36369,-0.373009,-0.742128,-0.687128,...,-0.056221,-0.482173,-0.258562,-1.535280,4.693590,-0.650811,-0.682500,-0.435643,-0.403480,3.277552
79669,-0.394003,-0.382911,1.967488,-0.278015,-0.400909,-0.853951,-0.36369,-0.373009,-0.742128,-0.687128,...,-0.056221,-0.482173,-0.258562,-1.535280,4.693590,-0.650811,-0.682500,-0.435643,-0.403480,3.277552
79670,-0.455410,-0.586511,1.967488,-0.278015,-0.359774,-0.853951,-0.36369,2.680870,-0.742128,-0.687128,...,-0.056221,-0.482173,-0.258562,0.651339,-0.213054,-0.650811,1.465184,-0.435643,-0.403480,-0.305102


In [13]:
def split_set(data_to_split, ratio=0.8):
    mask = np.random.rand(len(data_to_split)) < ratio
    return [data_to_split[mask].reset_index(drop=True), data_to_split[~mask].reset_index(drop=True)]

In [16]:
[train, test] = split_set(dummy_data)

### B) Train a logistic regression classifier on your training set. Logistic regression returns probabilities as predictions, so in order to arrive at a binary prediction, you need to put a threshold on the predicted probabilities. 
- For the decision threshold of 0.5, present the performance of your classifier on the test set by displaying the confusion matrix. Based on the confusion matrix, manually calculate accuracy, precision, recall, and F1-score with respect to the positive and the negative class. 

### C) Vary the value of the threshold in the range from 0 to 1 and visualize the value of accuracy, precision, recall, and F1-score (with respect to both classes) as a function of the threshold.

### D) Plot in a bar chart the coefficients of the logistic regression sorted by their contribution to the prediction.


## Question 1: Which of the following metrics is most suitable when you are dealing with unbalanced classes?

- a) F1 Score
- b) Recall
- c) Precision
- d) Accuracy

## Question 2: You are working on a binary classification problem. You trained a model on a training dataset and got the following confusion matrix on the test dataset. What is true about the evaluation metrics (rounded to the second decimal point):

|            | Pred = NO|Pred=YES|
|------------|----------|--------|
| Actual NO  |    50    |   10   |
| Actual YES |    5     |   100  |

- a) Accuracy is 0.95
- b) Accuracy is 0.85
- c) False positive rate is 0.95
- d) True positive rate is 0.95